<a href="https://colab.research.google.com/github/Sarah-0405/Coldspots_Bayern_Analyse/blob/main/LST_Dataframes_2019_2024_erstellen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


GEE Account und angelegtes Projekt erforderlich für Landsat-Daten-Bezug

In [3]:
# Importieren Sie notwendige Bibliotheken
import ee

In [5]:
ee.Authenticate()
ee.Initialize(project="ee-sarahheinz0405")

In [ ]:
!pip install pysal

# LST-Dataframes erstellen

wichtig: Wolkenmaskierung und Funktion für Umrechnung der LST

dataframes erstellen, die für jede Stadt die Pixelkoordinaten mitsamt deren LST-Wert enthalten
- für jedes Jahr
- für den 6-Jahres-Durchschnitt

aufgrund vorheriger Analyse wurde Cloud Cover threshold von <10% gewählt

In [8]:
import geopandas as gpd
from shapely.geometry import mapping, Point
import pandas as pd

In [9]:
# Laden Sie die Geometrien der Städte
staedte_ueber_50tsd_polygone = gpd.read_file("/content/drive/MyDrive/Cold Spots Bayern/grenzen_ueber_50tsd.gpkg")

# Definieren Sie die Jahre und den Zeitraum (Juni bis August)
years = range(2019, 2025) # Jahre 2019 bis einschließlich 2024
cloud_cover_threshold = 10

# Liste zum Speichern der GeoDataFrames für jede Stadt
all_cities_pixel_gdfs = []

# Funktion zur Wolkenmaskierung
def maskClouds(image):
    qa_band = image.select('QA_PIXEL')
    cloud_shadow_bitmask = (1 << 4)
    cloud_bitmask = (1 << 3)
    mask = qa_band.bitwiseAnd(cloud_shadow_bitmask).eq(0).And(
           qa_band.bitwiseAnd(cloud_bitmask).eq(0))
    return image.updateMask(mask)

# Funktion zur Berechnung der LST
def calculateLST(image):
    kelvin = image.select('ST_B10').multiply(0.00341802).add(149.0)
    lst_celsius = kelvin.subtract(273.15).rename('LST_Celsius')
    return image.addBands(lst_celsius)

# Funktion zum Extrahieren der Pixeldaten für eine einzelne Szene
def extract_pixels_from_image(image, region, current_year, city_name):
    try:
        scene_date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        # print(f"  Verarbeite Szene für {city_name} vom {scene_date}")

        masked_lst = image.select('LST_Celsius')
        mask = masked_lst.mask()
        lonLat_image = ee.Image.pixelLonLat()
        masked_lonLat = lonLat_image.updateMask(mask)
        array_image = masked_lst.addBands(masked_lonLat)

        pixel_data = array_image.reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=region,
            scale=30,
            maxPixels=1e9
        )

        pixel_data_info = pixel_data.getInfo()

        lst_values = pixel_data_info.get('LST_Celsius', [])
        lats = pixel_data_info.get('latitude', [])
        lons = pixel_data_info.get('longitude', [])

        if not lst_values or not lats or not lons or len(lst_values) != len(lats) or len(lst_values) != len(lons):
            print(f"    Warnung: Keine oder inkonsistente Pixeldaten für {city_name}, Jahr {current_year}, Szene {scene_date}. Skipping.")
            return []

        pixel_list = []
        for j in range(len(lst_values)):
            pixel_list.append({
                'city': city_name, # Stadtname hinzufügen
                'LST_Celsius': lst_values[j],
                'longitude': lons[j],
                'latitude': lats[j],
                'year': current_year,
                'scene_date': scene_date
            })
        return pixel_list
    except Exception as e:
        print(f"  Fehler bei der Verarbeitung der Szene für {city_name} vom {scene_date}: {e}")
        return []


# Schleife über jede Stadt im GeoDataFrame
for index, row in staedte_ueber_50tsd_polygone.iterrows():
    city_name = row['name']
    city_geometry_shapely = row['geometry']

    # Überprüfen Sie, ob die Geometrie gültig ist
    if not city_geometry_shapely or city_geometry_shapely.is_empty:
        print(f" Warnung: Ungültige oder leere Geometrie für Stadt: {city_name}. Überspringe diese Stadt.")
        continue

    # Konvertieren Sie die Shapely-Geometrie in eine Earth Engine Geometrie
    try:
        city_region_ee = ee.Geometry(mapping(city_geometry_shapely))
    except Exception as e:
        print(f"Fehler beim Erstellen der Earth Engine Geometrie für {city_name}: {e}. Überspringe Stadt.")
        continue

    print(f"\n Verarbeitung von Landsat-Daten für Stadt: {city_name}...")

    city_pixel_data_across_years = []

    # Schleife über jedes Jahr für die aktuelle Stadt
    for year in years:
        start_date = f'{year}-06-01'
        end_date = f'{year}-08-31'

        try:
            # Filtern, Maskieren und LST berechnen
            landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                .filterDate(start_date, end_date) \
                .filterBounds(city_region_ee) \
                .filter(ee.Filter.lt('CLOUD_COVER', cloud_cover_threshold)) \
                .map(maskClouds) \
                .map(calculateLST)

            num_scenes = landsat_collection.size().getInfo()
            print(f"    Anzahl der gefundenen Szenen: {num_scenes}") # Optional: Weniger Logs

            if num_scenes > 0:
                scenes_list = landsat_collection.toList(num_scenes)

                # Extrahieren Sie Pixeldaten für jede Szene im Zeitraum
                for i in range(num_scenes):
                    image = ee.Image(scenes_list.get(i))
                    current_year = year

                    current_scene_pixel_data = extract_pixels_from_image(image, city_region_ee, current_year, city_name)
                    city_pixel_data_across_years.extend(current_scene_pixel_data)
            else:
                print(f"    Keine Szenen für {city_name} im Jahr {year} gefunden.") # Optional: Weniger Logs

        except Exception as e:
            print(f"  Fehler bei der Verarbeitung der Landsat-Collection für {city_name} im Jahr {year}: {e}")

    # Wenn Daten für die aktuelle Stadt gesammelt wurden, erstellen Sie den GeoDataFrame
    if city_pixel_data_across_years:
        df_city_pixels_all_years = pd.DataFrame(city_pixel_data_across_years)

        # Erstellen Sie die Geometrie-Spalte
        geometry_all_years = [Point(xy) for xy in zip(df_city_pixels_all_years['longitude'], df_city_pixels_all_years['latitude'])]
        gdf_city_pixels_all_years = gpd.GeoDataFrame(df_city_pixels_all_years, geometry=geometry_all_years)
        gdf_city_pixels_all_years.crs = "EPSG:4326" # Setzen Sie das CRS

        print(f"\nGeoDataFrame für {city_name} mit Pixel-LST für alle Jahre (2019-2024) erstellt. Enthält {len(gdf_city_pixels_all_years)} Pixel.")
        # display(gdf_city_pixels_all_years.head()) # Optional: Die ersten Zeilen anzeigen

        # Füge den erstellten GeoDataFrame der Liste hinzu
        all_cities_pixel_gdfs.append(gdf_city_pixels_all_years)

        #  Speichern Sie den GeoDataFrame für jede Stadt einzeln
        #output_city_geojson_path = f"/content/drive/MyDrive/Cold Spots Bayern/{city_name.replace(' ', '_')}_pixels_lst_2019_2024.geojson"
        #try:
             #gdf_city_pixels_all_years.to_file(output_city_geojson_path, driver='GeoJSON')
             #print(f"  GeoDataFrame für {city_name} wurde gespeichert unter: {output_city_geojson_path}")
        #except Exception as e:
             #print(f"  Fehler beim Speichern des GeoDataFrames für {city_name}: {e}")

    else:
        print(f"Keine Pixeldaten für Stadt {city_name} gefunden.")



 Verarbeitung von Landsat-Daten für Stadt: Munich...
    Anzahl der gefundenen Szenen: 6
    Anzahl der gefundenen Szenen: 4
    Anzahl der gefundenen Szenen: 1
    Anzahl der gefundenen Szenen: 6
    Anzahl der gefundenen Szenen: 3
    Anzahl der gefundenen Szenen: 3

GeoDataFrame für Munich mit Pixel-LST für alle Jahre (2019-2024) erstellt. Enthält 6733177 Pixel.

 Verarbeitung von Landsat-Daten für Stadt: Nuremberg...
    Anzahl der gefundenen Szenen: 5


    Anzahl der gefundenen Szenen: 4
    Anzahl der gefundenen Szenen: 2
    Anzahl der gefundenen Szenen: 5
    Anzahl der gefundenen Szenen: 1
    Anzahl der gefundenen Szenen: 2

GeoDataFrame für Nuremberg mit Pixel-LST für alle Jahre (2019-2024) erstellt. Enthält 3691492 Pixel.

 Verarbeitung von Landsat-Daten für Stadt: Augsburg...
    Anzahl der gefundenen Szenen: 7
    Anzahl der gefundenen Szenen: 4
    Anzahl der gefundenen Szenen: 2
    Anzahl der gefundenen Szenen: 7
    Anzahl der gefundenen Szenen: 2
    Anzahl der gefundenen Szenen: 2

GeoDataFrame für Augsburg mit Pixel-LST für alle Jahre (2019-2024) erstellt. Enthält 3432603 Pixel.

 Verarbeitung von Landsat-Daten für Stadt: Regensburg...
    Anzahl der gefundenen Szenen: 4
    Anzahl der gefundenen Szenen: 3
    Anzahl der gefundenen Szenen: 1
    Anzahl der gefundenen Szenen: 3
    Anzahl der gefundenen Szenen: 2
    Anzahl der gefundenen Szenen: 2

GeoDataFrame für Regensburg mit Pixel-LST für alle Jahre (2019-2024) e

    Anzahl der gefundenen Szenen: 2

GeoDataFrame für Würzburg mit Pixel-LST für alle Jahre (2019-2024) erstellt. Enthält 887149 Pixel.

 Verarbeitung von Landsat-Daten für Stadt: Fürth...
    Anzahl der gefundenen Szenen: 5
    Anzahl der gefundenen Szenen: 4
    Anzahl der gefundenen Szenen: 2
    Anzahl der gefundenen Szenen: 5
    Anzahl der gefundenen Szenen: 1
    Anzahl der gefundenen Szenen: 2

GeoDataFrame für Fürth mit Pixel-LST für alle Jahre (2019-2024) erstellt. Enthält 1072747 Pixel.

 Verarbeitung von Landsat-Daten für Stadt: Erlangen...
    Anzahl der gefundenen Szenen: 7
    Anzahl der gefundenen Szenen: 4
    Anzahl der gefundenen Szenen: 2
    Anzahl der gefundenen Szenen: 6
    Anzahl der gefundenen Szenen: 2
    Anzahl der gefundenen Szenen: 3

GeoDataFrame für Erlangen mit Pixel-LST für alle Jahre (2019-2024) erstellt. Enthält 983613 Pixel.

 Verarbeitung von Landsat-Daten für Stadt: Bamberg...
    Anzahl der gefundenen Szenen: 6
    Anzahl der gefundenen Szenen: 

läuft jetzt erfolgreich für alle Städte durch (ca. 20 min) und speichert für die Sommermonate 2019-2024 alle LST-Werte pro Pixel pro Szene in einem gdf pro Stadt

In [10]:
cities_paths = "/content/drive/MyDrive/Cold Spots Bayern/Aschaffenburg_pixels_lst_2019_2024.geojson", "/content/drive/MyDrive/Cold Spots Bayern/Augsburg_pixels_lst_2019_2024.geojson", "/content/drive/MyDrive/Cold Spots Bayern/Bamberg_pixels_lst_2019_2024.geojson", "/content/drive/MyDrive/Cold Spots Bayern/Bayreuth_pixels_lst_2019_2024.geojson", "/content/drive/MyDrive/Cold Spots Bayern/Erlangen_pixels_lst_2019_2024.geojson", "/content/drive/MyDrive/Cold Spots Bayern/Fürth_pixels_lst_2019_2024.geojson", "/content/drive/MyDrive/Cold Spots Bayern/Ingolstadt_pixels_lst_2019_2024.geojson", "/content/drive/MyDrive/Cold Spots Bayern/Kempten_(Allgäu)_pixels_lst_2019_2024.geojson", "/content/drive/MyDrive/Cold Spots Bayern/Landshut_pixels_lst_2019_2024.geojson", "/content/drive/MyDrive/Cold Spots Bayern/Munich_pixels_lst_2019_2024.geojson", "/content/drive/MyDrive/Cold Spots Bayern/Nuremberg_pixels_lst_2019_2024.geojson", "/content/drive/MyDrive/Cold Spots Bayern/Passau_pixels_lst_2019_2024.geojson", "/content/drive/MyDrive/Cold Spots Bayern/Regensburg_pixels_lst_2019_2024.geojson", "/content/drive/MyDrive/Cold Spots Bayern/Rosenheim_pixels_lst_2019_2024.geojson", "/content/drive/MyDrive/Cold Spots Bayern/Schweinfurt_pixels_lst_2019_2024.geojson", "/content/drive/MyDrive/Cold Spots Bayern/Würzburg_pixels_lst_2019_2024.geojson"

bisher enthalten geodataframes Daten zur jeder einzelnen Szene =>  daraus den Mittelwert pro Pixel über ganzen Sommer berechnen (pro Jahr), dann zusammenführen zu einer Datei für alle Städte

In [ ]:
# Die Liste der GeoJSON-Dateipfade (aus der cities_paths Zelle)
# cities_paths = [...] # Angenommen, diese Variable ist bereits definiert

# Liste zum Speichern der aggregierten GeoDataFrames für jede Stadt
aggregated_city_gdfs = []

print("Verarbeite jede Stadt einzeln und berechne die durchschnittliche Sommer-LST pro Pixel pro Jahr...")

# Schleife über jeden Dateipfad in der cities_paths Liste
for file_path in cities_paths:
    try:
        # Lade den GeoDataFrame für die aktuelle Stadt
        # Versuche, den Stadtnamen aus dem Dateipfad zu extrahieren
        file_name = file_path.split('/')[-1]
        city_name = file_name.replace("_pixels_lst_2019_2024.geojson", "").replace("_", " ")

        print(f"\n Verarbeite Daten für Stadt: {city_name} aus {file_name}...")
        gdf_city = gpd.read_file(file_path)
        print(f"  Geladen: {len(gdf_city)} Pixel-Szene-Einträge.")

        # Überprüfe, ob der GeoDataFrame geladen wurde und nicht leer ist
        if not gdf_city.empty:
            # Gruppieren Sie nach Jahr, Längengrad und Breitengrad und berechnen Sie den Durchschnitt der LST
            # Behalten Sie 'city' als Teil der Gruppierung bei
            aggregated_lst_city = gdf_city.groupby(
                ['year', 'longitude', 'latitude', 'city'] # Gruppieren auch nach Stadt
            )['LST_Celsius'].mean().reset_index()

            # Benennen Sie die aggregierte Spalte um
            aggregated_lst_city = aggregated_lst_city.rename(
                columns={'LST_Celsius': 'avg_summer_LST_Celsius'}
            )

            # Fügen Sie die Geometrie-Spalte wieder hinzu
            unique_pixels_geometry_city = gdf_city[['longitude', 'latitude', 'geometry']].drop_duplicates(
                subset=['longitude', 'latitude']
            ).set_index(['longitude', 'latitude'])

            aggregated_lst_city = aggregated_lst_city.set_index(
                ['longitude', 'latitude']
            ).join(unique_pixels_geometry_city).reset_index()

            # Konvertieren Sie das Ergebnis in einen GeoDataFrame
            # Stellen Sie sicher, dass das CRS des ursprünglichen GeoDataFrames übernommen wird
            if gdf_city.crs is not None:
                gdf_avg_summer_lst_city = gpd.GeoDataFrame(
                    aggregated_lst_city,
                    geometry='geometry',
                    crs=gdf_city.crs
                )
            else:
                print(f"   Warnung: Ursprüngliches CRS für {city_name} nicht gefunden. Setze Standard-CRS (EPSG:4326).")
                gdf_avg_summer_lst_city = gpd.GeoDataFrame(
                    aggregated_lst_city,
                    geometry='geometry',
                    crs="EPSG:4326"
                )


            print(f"  Erstellt aggregierten GeoDataFrame für {city_name} mit {len(gdf_avg_summer_lst_city)} Pixel-Jahres-Einträgen.")

            # Fügen Sie den aggregierten GeoDataFrame der Liste hinzu
            aggregated_city_gdfs.append(gdf_avg_summer_lst_city)


        else:
            print(f"  GeoDataFrame für {city_name} war leer. Überspringe Aggregation.")

    except Exception as e:
        print(f"  Fehler bei der Verarbeitung der Datei {file_path}: {e}. Überspringe diese Datei.")


# Zusammenführen der aggregierten Stadt-GeoDataFrames
print("\n--- Zusammenführen der aggregierten Stadt-GeoDataFrames ---")

if aggregated_city_gdfs:
    try:
        all_cities_avg_summer_lst_per_pixel_year = pd.concat(aggregated_city_gdfs, ignore_index=True)

        print("\nZusammengeführter GeoDataFrame 'all_cities_avg_summer_lst_per_pixel_year' erstellt:")
        display(all_cities_avg_summer_lst_per_pixel_year.head())
        print(f"Gesamtanzahl der Pixel-Jahres-Einträge im zusammengeführten GeoDataFrame: {len(all_cities_avg_summer_lst_per_pixel_year)}")

        # Speichern des finalen zusammengeführten GeoDataFrame
        output_final_merged_geojson_path = "/content/drive/MyDrive/Cold Spots Bayern/all_cities_summer_avg_lst_per_pixel_allyears.geojson"
        try:
             all_cities_avg_summer_lst_per_pixel_year.to_file(output_final_merged_geojson_path, driver='GeoJSON')
             print(f"\nFinaler zusammengeführter GeoDataFrame wurde gespeichert unter: {output_final_merged_geojson_path}")
        except Exception as e:
             print(f"\nFehler beim Speichern des finalen zusammengeführten GeoDataFrames: {e}")

    except Exception as e:
        print(f"Fehler beim Zusammenführen der aggregierten GeoDataFrames: {e}")

else:
    print("Keine aggregierten Stadt-GeoDataFrames erstellt. Zusammenführung nicht möglich.")


Verarbeite jede Stadt einzeln und berechne die durchschnittliche Sommer-LST pro Pixel pro Jahr...

 Verarbeite Daten für Stadt: Aschaffenburg aus Aschaffenburg_pixels_lst_2019_2024.geojson...
  Geladen: 525955 Pixel-Szene-Einträge.
  Erstellt aggregierten GeoDataFrame für Aschaffenburg mit 362052 Pixel-Jahres-Einträgen.

 Verarbeite Daten für Stadt: Augsburg aus Augsburg_pixels_lst_2019_2024.geojson...
  Geladen: 3531413 Pixel-Szene-Einträge.
  Erstellt aggregierten GeoDataFrame für Augsburg mit 980443 Pixel-Jahres-Einträgen.

 Verarbeite Daten für Stadt: Bamberg aus Bamberg_pixels_lst_2019_2024.geojson...
  Geladen: 895498 Pixel-Szene-Einträge.
  Erstellt aggregierten GeoDataFrame für Bamberg mit 543528 Pixel-Jahres-Einträgen.

 Verarbeite Daten für Stadt: Bayreuth aus Bayreuth_pixels_lst_2019_2024.geojson...
  Geladen: 594897 Pixel-Szene-Einträge.
  Erstellt aggregierten GeoDataFrame für Bayreuth mit 521674 Pixel-Jahres-Einträgen.

 Verarbeite Daten für Stadt: Erlangen aus Erlangen_p

output: geojson mit den mittleren LST-Pixelwerten **für jedes Jahr** von 2019
-2024: "all_cities_summer_avg_lst_per_pixel_allyears.geojson"

# Geodataframes für 2019-2024 sowie 2024 separat erstellen

- ein gdf mit Durchschnittswerten pro Pixel für Gesamtzeitraum,
- einer mit Durchschnittswerten für 2024

In [ ]:
# oben erstellte Datei laden (enthält noch die Werte für jedes Jahr)
all_cities_summer_avg_lst_per_pixel = gpd.read_file("/content/drive/MyDrive/Cold Spots Bayern/all_cities_summer_avg_lst_per_pixel_allyears.geojson")
display(all_cities_summer_avg_lst_per_pixel.head())

In [ ]:
# 1. Geodataframe 2019-2024 erstellen

# Gruppierung und Mittelwertbildung
# Gruppieren nach 'city' und 'geometry' und berechnen des Mittelwerts für 'avg_summer_LST_Celsius'
# 'geometry' als Index, um die Gruppierung zu vereinfachen und dann zurückzusetzen
averaged_lst_per_pixel_per_city = all_cities_summer_avg_lst_per_pixel.set_index('geometry').groupby(['city', 'geometry'])['avg_summer_LST_Celsius'].mean().reset_index()

# Verwenden Sie das CRS des ursprünglichen GeoDataFrames
original_crs = all_cities_summer_avg_lst_per_pixel.crs

averaged_lst_per_pixel_per_city = gpd.GeoDataFrame(
    averaged_lst_per_pixel_per_city,
    geometry='geometry',
    crs=original_crs
)

print("Mittelwert der LST pro Pixel pro Stadt über alle Jahre berechnet.")
display(averaged_lst_per_pixel_per_city.head())

In [ ]:
# Gdf 2019-2024 speichern

# Definieren Sie den Pfad zum Speichern der Datei in Google Drive
# Passen Sie den Ordner und Dateinamen bei Bedarf an
output_path_averaged_gdf = "/content/drive/MyDrive/Cold Spots Bayern/averaged_lst_per_pixel_per_city_allyears.geojson"

try:
    # Speichern als GeoJSON-Datei
    averaged_lst_per_pixel_per_city.to_file(output_path_averaged_gdf, driver='GeoJSON')
    print(f"GeoDataFrame mit gemittelten LST-Werten erfolgreich gespeichert unter: {output_path_averaged_gdf}")
except Exception as e:
    print(f"Fehler beim Speichern des GeoDataFrames: {e}")

In [ ]:
# 2. Geodataframe für 2024 erstellen

# Filter the geodataframe for the year 2024
gdf_2024 = all_cities_summer_avg_lst_per_pixel[all_cities_summer_avg_lst_per_pixel['year'] == 2024].copy()

print(gdf_2024.head())

In [ ]:
# gdf 2024 speichern

output_path = "/content/drive/MyDrive/Cold Spots Bayern/lst_2024_allcities.geojson"
gdf_2024.to_file(output_path, driver='GeoJSON')

print(f"Geodataframe 'gdf_2024' erfolgreich gespeichert als: {output_path}")

output: mit den geojson Dateien "lst_2024_allcities.geojson" und "averaged_lst_per_pixel_per_city_allyears.geojson" kann jetzt weitergearbeitet werden

nächste Schritte: Cold Spots berechnen mittels KNN und Gi* Algorithmus

### Visualisierung der LST-Verteilung pro Stadt auf Karte

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
# 2019-2024 Durschnittsdaten laden und visualisieren

file_path = "/content/drive/MyDrive/Cold Spots Bayern/averaged_lst_per_pixel_per_city_allyears.geojson"
lst_data = gpd.read_file(file_path)
display(lst_data.head())


In [ ]:
# Daten nach Städten aufteilen

# Check unique cities
unique_cities = lst_data['city'].unique()
print(f"Found data for the following cities: {unique_cities}")

# Create a dictionary to hold GeoDataFrames for each city
city_lst_gdfs = {}
for city in unique_cities:
    city_lst_gdfs[city] = lst_data[lst_data['city'] == city].copy()
    print(f"Created GeoDataFrame for {city} with {len(city_lst_gdfs[city])} features.")

# Visualisierung für jede Stadt

cmap_cold_hot = LinearSegmentedColormap.from_list("cold_to_hot", ["blue", "cyan", "yellow", "red"])


print("\nGenerating maps for each city...")

for city, gdf in city_lst_gdfs.items():
    if not gdf.empty:
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        # Plot the data, mapping 'avg_summer_LST_Celsius' to color
        # Use the custom colormap and add a colorbar
        gdf.plot(column='avg_summer_LST_Celsius', ax=ax, legend=True,
                 cmap=cmap_cold_hot,
                 legend_kwds={'label': "Average Summer LST (°C)",
                              'orientation': "horizontal"})

        ax.set_title(f'Average Summer LST for {city}')
        ax.set_axis_off()
        #plt.show() # unkommentieren, dann werden Karten angezeigt
    else:
        print(f"No data to plot for {city}. Skipping map generation.")

print("\nMap generation complete.")
